In [ ]:
import os
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import numpy as np
 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.compose import ColumnTransformer
from scipy.stats import norm
import scipy.stats as stats
import numpy as np

---
# Inimse hingamise sageduse ja teiste füüsiliste parameetrite regressioonanalüüs
## ITB8814 Andmekaevandamine, Projekt
### Autor: Juri Lunin
Kuupäev 17.05.2024

<style>
body p {
  text-align: justify
}
</style>

# Sissejuhatus

<p>
Käesolevas projektitöö on valitud uurimiseks andmestik: "Energy Expenditure of Human Physical Activity".
</p>

## Andmestiku kirjeldus

Andmestikus on inimeste füüsilised omadused ja info nende seisundi kohta, mis avaldub füüsilise aktiivsuse ajal.
<a href="https://www.kaggle.com/datasets/anonymousds/energy-expenditure-of-human-physical-activity">Energy Expenditure of Human Physical Activity</a>  Faili formaat: csv*.

Andmestikuga koos on postitatud kaks teadusliku artikli:

<p>

<ul>

<li><a href="https://link.springer.com/article/10.1007/s11257-020-09268-2">Activity recognition using wearable sensors for tracking the elderly</a></li>

<li><a href="https://link.springer.com/article/10.1007/s10618-021-00817-w">A recurrent neural network architecture to model physical activity energy expenditure in older people</a></li>

</ul>

</p>

Andmestiku tunnused:

<ul>
<li>ID - participant's ID</li>
<li>trial_date - date and time when data collection started at ID level</li>
<li>gender - sex = male or female</li>
<li>age	in years</li>
<li>weight	in kg</li>
<li>height	in cm</li>
<li>bmi	Body mass index in kg/m</li>
<li>gaAnkle	TRUE if data from GENEActiv on the ankle exist, FALSE otherwise</li>
<li>gaChest	TRUE if data from GENEActiv on the chest exist, FALSE otherwise</li>
<li>gaWrist	TRUE if data from GENEActiv on the wrist exist, FALSE otherwise</li>
<li>equivital	TRUE if data from Equivital exist, FALSE otherwise</li>
<li>cosmed	TRUE if data from COSMED exist, FALSE otherwise</li>
<li>EEm	Energy Expenditure per minute, in Kcal</li>
<li>COSMEDset_row	the original indexes of COSMED data (used for merging)</li>
<li>EEh	Energy Expenditure per hour, in Kcal</li>
<li>EEtot	Total Kcal spent (it is reseted between indoor and outdoor measurements)</li>
<li>METS	Metabolic Equivalent per minute</li>
<li>Rf	Respiratory Frequency (litre/min)</li>
<li>BR	Breath Rate</li>
<li>VT	Tidal Volume in litre</li>
<li>VE	Expiratory Minute Ventilation (litre/min)</li>
<li>VO2	Oxygen Uptake (ml/min)</li>
<li>VCO2	Carbon Dioxide production (ml/min)</li>
<li>O2exp	Volume of O2 expired (ml/min)</li>
<li>CO2exp	Volume of CO2 expired (ml/min)</li>
<li>FeO2	Averaged expiratory concentration of O2 (%)</li>
<li>FeCO2	Averaged expiratory concentration of CO2 (%)</li>
<li>FiO2	Fraction of inspired O2 (%)</li>
<li>FiCO2	Fraction of inspired CO2 (%)</li>
<li>VE.VO2	Ventilatory equivalent for O2</li>
<li>VE.VCO2	Ventilatory equivalent for CO2</li>
<li>R	Respiratory Quotient</li>
<li>Ti	Duration of Inspiration (seconds)</li>
<li>Te	Duration of Expiration (seconds)</li>
<li>Ttot	Duration of Total breathing cycle (seconds)</li>
<li>VO2.HR	Oxygen pulse (ml/beat)</li>
<li>HR	Heart Rate</li>
<li>Qt	Cardiac output (litre)</li>
<li>SV	Stroke volume (litre/min)</li>
<li>original_activity_labels	True activity label as noted from study protocol, NA if is unknown</li>
<li>predicted_activity_label	Predicted activity label by model from [1], NA if is unknown</li>
</ul>

</p>

## Uurimiseesmärk

Prognoositav tunnus ehk sihttunnus on Y=“BR” breath rate.

Eesmärk: parima mitme argumendiga regressiooni mudeli leidmine sihttunnuse *BR* prognoosimiseks.

## Tööülesanded (tööhüpoteesid)

1. Madalama hingamise sagedusega inimestel on tugevam tervis.

## Metoodika ja uuringu käik

Käesoleva projektis kasutasime järgnevaid analüüsi meetodeid:

<ul>
<li>Lineaarregressioon</li>
<li>Polünomiaalregressioon</li>
<li>Otsustuspuu regressiooni mudel</li>
<li>Random forest regressiooni mudel</li>
</ul>